# Model Building and Verification

## 1. Modules and Libraries

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')


## 2. Load Data

1. Read the CSV file and display few rows
2. Split the data into features and labels
3. Display the shape of the data

In [2]:
# Read the data
promapen_train = pd.read_csv("features/ProMapEn/promapen_train_similarities.csv")
promapen_test = pd.read_csv("features/ProMapEn/promapen_test_similarities.csv")

promapen_train.head()

,name_cos,short_description_cos,long_description_cos,all_texts_cos,name_id,short_description_id,all_texts_id,name_brand,short_description_brand,all_texts_brand,...,short_description_words,all_texts_words,all_units_list,all_ids_list,all_numbers_list,all_brands_list,specification_key,specification_key_value,hash_similarity,match
0,0.171232,0.059608,0.116891,0.466738,0.2,0.142857,0.194444,0.0,0.0,1.0,...,0.068966,0.369369,0.148148,0.169492,0.162162,0.666667,0.0,0.0,0.0,0
1,0.119147,0.248724,0.258786,0.570979,0.0,0.050000,0.267857,0.0,0.0,0.0,...,0.151786,0.455752,0.277778,0.237624,0.307692,0.000000,0.0,0.0,0.0,0
2,0.077348,0.454257,0.659632,0.627801,0.0,0.466667,0.195122,0.0,0.0,0.0,...,0.493333,0.423611,0.083333,0.259259,0.133333,0.000000,0.0,0.0,0.0,0
3,0.531188,0.243689,0.200950,0.608172,1.0,0.250000,0.500000,0.0,0.0,0.0,...,0.096774,0.473118,0.285714,0.241379,0.250000,0.000000,0.0,0.0,0.0,1
4,0.905550,0.000000,0.244415,0.872226,0.5,0.000000,0.666667,0.0,0.0,0.0,...,0.000000,0.821782,0.466667,0.411765,0.444444,0.000000,0.0,0.0,0.0,0


In [3]:
# Split the data into features and labels
X_train_promapen = promapen_train.iloc[:, :-1]
y_train_promapen = promapen_train.iloc[:, -1]

X_test_promapen = promapen_test.iloc[:, :-1]
y_test_promapen = promapen_test.iloc[:, -1]

print(f"ProMapEn Train data: {X_train_promapen.shape}")
print(f"ProMapEn Train label: {y_train_promapen.shape}")
print(f"ProMapEn Test data: {X_test_promapen.shape}")
print(f"ProMapEn Test label: {y_test_promapen.shape}")


ProMapEn Train data: (1244, 34)
ProMapEn Train label: (1244,)
ProMapEn Test data: (311, 34)
ProMapEn Test label: (311,)


## 3. Random and Grid Search CV

* base_model_id = 0 for Randomized Search CV
* base_model_id = 1 for Grid Search CV

In [4]:
class RandomAndGridSearchCV:
    """Extension of Randomized and Grid Search to accomodate various models"""

    def __init__(self, X_train, y_train, X_test, y_test, base_model_id, classifier, params, n_iter=10, n_jobs=None, verbose=10):
        """
        Initialize the models
        Args:
            X_train (pd.DataFrame): train data
            y_train (pd.DataFrame): train labels
            X_test (pd.DataFrame): test data
            y_test (pd.DataFrame): test labels
            base_model_id (int): 0 => Randomized Search CV (OR) 1 => Grid Search CV
            classifier (sklearn): machine learning model
            params (dict): CV search parameters
        """
        self.base_model_id = base_model_id
        self.estimator = classifier
        self.params = params
        self.n_iter = n_iter
        self.n_jobs = n_jobs
        self.verbose = verbose
        self.best_params = None
        self.best_model = None

        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test

        self.accuracy = None
        self.f1 = None
        self.precision = None
        self.recall = None
        
        self.args = {
            "estimator": self.estimator(random_state=42),
            "cv": 5,
            "scoring": {
                "accuracy": make_scorer(accuracy_score),
                "f1": make_scorer(f1_score), 
                "precision": make_scorer(precision_score), 
                "recall": make_scorer(recall_score)
            },
            "refit": "f1",
            "n_jobs": self.n_jobs,
            "verbose": self.verbose
        }
         
    
    def set_cv_model_and_params(self):
        """
        Set the base model to Grid Search or Randomized CV
        """
        
        if self.base_model_id==0:
            self.base_model = RandomizedSearchCV
            self.args["param_distributions"] = self.params
            self.args["n_iter"] = self.n_iter
        
        elif self.base_model_id==1:
            self.base_model = GridSearchCV
            self.args["param_grid"] = self.params
    
    def estimate_best_params(self):
        """
        Estimate the best parameters using grid search
        """
        self.set_cv_model_and_params()
        
        search_cv = self.base_model(**self.args)

        search_cv.fit(self.X_train, self.y_train)

        self.best_params = search_cv.best_params_
        
        self.f1 = search_cv.best_score_
        self.precision = search_cv.cv_results_['mean_test_precision'][search_cv.best_index_]
        self.recall = search_cv.cv_results_['mean_test_recall'][search_cv.best_index_]
        self.accuracy = search_cv.cv_results_['mean_test_accuracy'][search_cv.best_index_]



### 3.1 Logistic Regression

In [5]:
# search CV parameters for logisitic regression
logistic_param_grid = {
    'penalty':["l1", "l2", "elasticnet", "none"],
    'solver': ["lbfgs", "newton-cg", "liblinear", "sag", "saga"],
    'max_iter': [10, 20, 50, 100, 200, 500]
}


#### 3.1.1 Randomised Search CV - Logistic Regression

In [6]:
# initialize the logistic regression
lg_rand = RandomAndGridSearchCV(X_train=X_train_promapen,
                                y_train=y_train_promapen,
                                X_test=X_test_promapen,
                                y_test=y_test_promapen,
                                base_model_id=0,
                                classifier=LogisticRegression,
                                params=logistic_param_grid, 
                                n_iter=10)

# estimate the best parameters for logisitic regression
lg_rand.estimate_best_params()

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5; 1/10] START max_iter=50, penalty=l2, solver=newton-cg..................
[CV 1/5; 1/10] END max_iter=50, penalty=l2, solver=newton-cg; accuracy: (test=0.763) f1: (test=0.587) precision: (test=0.677) recall: (test=0.519) total time=   0.0s
[CV 2/5; 1/10] START max_iter=50, penalty=l2, solver=newton-cg..................
[CV 2/5; 1/10] END max_iter=50, penalty=l2, solver=newton-cg; accuracy: (test=0.763) f1: (test=0.504) precision: (test=0.811) recall: (test=0.366) total time=   0.0s
[CV 3/5; 1/10] START max_iter=50, penalty=l2, solver=newton-cg..................
[CV 3/5; 1/10] END max_iter=50, penalty=l2, solver=newton-cg; accuracy: (test=0.711) f1: (test=0.478) precision: (test=0.589) recall: (test=0.402) total time=   0.0s
[CV 4/5; 1/10] START max_iter=50, penalty=l2, solver=newton-cg..................
[CV 4/5; 1/10] END max_iter=50, penalty=l2, solver=newton-cg; accuracy: (test=0.751) f1: (test=0.575) precision: (tes

[CV 3/5; 8/10] END max_iter=200, penalty=none, solver=sag; accuracy: (test=0.711) f1: (test=0.520) precision: (test=0.574) recall: (test=0.476) total time=   0.1s
[CV 4/5; 8/10] START max_iter=200, penalty=none, solver=sag.....................
[CV 4/5; 8/10] END max_iter=200, penalty=none, solver=sag; accuracy: (test=0.719) f1: (test=0.557) precision: (test=0.579) recall: (test=0.537) total time=   0.1s
[CV 5/5; 8/10] START max_iter=200, penalty=none, solver=sag.....................
[CV 5/5; 8/10] END max_iter=200, penalty=none, solver=sag; accuracy: (test=0.766) f1: (test=0.586) precision: (test=0.695) recall: (test=0.506) total time=   0.1s
[CV 1/5; 9/10] START max_iter=100, penalty=none, solver=liblinear...............
[CV 1/5; 9/10] END max_iter=100, penalty=none, solver=liblinear; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=   0.0s
[CV 2/5; 9/10] START max_iter=100, penalty=none, solver=liblinear...............
[CV 2/5; 9/10] END max_ite

In [7]:
# Build the model with best parameters and test
print(f"----- RANDOMIZED SEARCH CV FOR LOGISTIC REGRESSION -----")
print(f"Best Parameters {lg_rand.best_params}")
print(f"F1 Score {lg_rand.f1}")
print(f"Precision Score {lg_rand.precision}")
print(f"Recall Score {lg_rand.recall}")
print(f"Accuracy Score {lg_rand.accuracy}")

----- RANDOMIZED SEARCH CV FOR LOGISTIC REGRESSION -----
Best Parameters {'solver': 'sag', 'penalty': 'none', 'max_iter': 200}
F1 Score 0.5576341881530109
Precision Score 0.6694784068846092
Recall Score 0.4878048780487805
Accuracy Score 0.7451936779375566


#### 3.1.2 Grid Search CV - Logistic Regression

In [8]:
lg_grid = RandomAndGridSearchCV(X_train=X_train_promapen,
                                y_train=y_train_promapen,
                                X_test=X_test_promapen,
                                y_test=y_test_promapen,
                                base_model_id=1,
                                classifier=LogisticRegression,
                                params=logistic_param_grid)

lg_grid.estimate_best_params()

Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5; 1/120] START max_iter=10, penalty=l1, solver=lbfgs.....................
[CV 1/5; 1/120] END max_iter=10, penalty=l1, solver=lbfgs; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=   0.0s
[CV 2/5; 1/120] START max_iter=10, penalty=l1, solver=lbfgs.....................
[CV 2/5; 1/120] END max_iter=10, penalty=l1, solver=lbfgs; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=   0.0s
[CV 3/5; 1/120] START max_iter=10, penalty=l1, solver=lbfgs.....................
[CV 3/5; 1/120] END max_iter=10, penalty=l1, solver=lbfgs; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=   0.0s
[CV 4/5; 1/120] START max_iter=10, penalty=l1, solver=lbfgs.....................
[CV 4/5; 1/120] END max_iter=10, penalty=l1, solver=lbfgs; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time= 

[CV 3/5; 8/120] END max_iter=10, penalty=l2, solver=liblinear; accuracy: (test=0.715) f1: (test=0.482) precision: (test=0.600) recall: (test=0.402) total time=   0.0s
[CV 4/5; 8/120] START max_iter=10, penalty=l2, solver=liblinear.................
[CV 4/5; 8/120] END max_iter=10, penalty=l2, solver=liblinear; accuracy: (test=0.755) f1: (test=0.585) precision: (test=0.662) recall: (test=0.524) total time=   0.0s
[CV 5/5; 8/120] START max_iter=10, penalty=l2, solver=liblinear.................
[CV 5/5; 8/120] END max_iter=10, penalty=l2, solver=liblinear; accuracy: (test=0.770) f1: (test=0.558) precision: (test=0.750) recall: (test=0.444) total time=   0.0s
[CV 1/5; 9/120] START max_iter=10, penalty=l2, solver=sag.......................
[CV 1/5; 9/120] END max_iter=10, penalty=l2, solver=sag; accuracy: (test=0.771) f1: (test=0.596) precision: (test=0.700) recall: (test=0.519) total time=   0.0s
[CV 2/5; 9/120] START max_iter=10, penalty=l2, solver=sag.......................
[CV 2/5; 9/120

[CV 1/5; 17/120] END max_iter=10, penalty=none, solver=newton-cg; accuracy: (test=0.739) f1: (test=0.552) precision: (test=0.625) recall: (test=0.494) total time=   0.0s
[CV 2/5; 17/120] START max_iter=10, penalty=none, solver=newton-cg..............
[CV 2/5; 17/120] END max_iter=10, penalty=none, solver=newton-cg; accuracy: (test=0.791) f1: (test=0.574) precision: (test=0.875) recall: (test=0.427) total time=   0.0s
[CV 3/5; 17/120] START max_iter=10, penalty=none, solver=newton-cg..............
[CV 3/5; 17/120] END max_iter=10, penalty=none, solver=newton-cg; accuracy: (test=0.711) f1: (test=0.520) precision: (test=0.574) recall: (test=0.476) total time=   0.0s
[CV 4/5; 17/120] START max_iter=10, penalty=none, solver=newton-cg..............
[CV 4/5; 17/120] END max_iter=10, penalty=none, solver=newton-cg; accuracy: (test=0.719) f1: (test=0.557) precision: (test=0.579) recall: (test=0.537) total time=   0.0s
[CV 5/5; 17/120] START max_iter=10, penalty=none, solver=newton-cg...........

[CV 5/5; 23/120] END max_iter=20, penalty=l1, solver=liblinear; accuracy: (test=0.766) f1: (test=0.547) precision: (test=0.745) recall: (test=0.432) total time=   0.0s
[CV 1/5; 24/120] START max_iter=20, penalty=l1, solver=sag......................
[CV 1/5; 24/120] END max_iter=20, penalty=l1, solver=sag; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=   0.0s
[CV 2/5; 24/120] START max_iter=20, penalty=l1, solver=sag......................
[CV 2/5; 24/120] END max_iter=20, penalty=l1, solver=sag; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=   0.0s
[CV 3/5; 24/120] START max_iter=20, penalty=l1, solver=sag......................
[CV 3/5; 24/120] END max_iter=20, penalty=l1, solver=sag; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=   0.0s
[CV 4/5; 24/120] START max_iter=20, penalty=l1, solver=sag......................
[CV 4/5; 24/120] END max_iter=20, penalty=l1, 

[CV 3/5; 37/120] END max_iter=20, penalty=none, solver=newton-cg; accuracy: (test=0.711) f1: (test=0.520) precision: (test=0.574) recall: (test=0.476) total time=   0.0s
[CV 4/5; 37/120] START max_iter=20, penalty=none, solver=newton-cg..............
[CV 4/5; 37/120] END max_iter=20, penalty=none, solver=newton-cg; accuracy: (test=0.719) f1: (test=0.557) precision: (test=0.579) recall: (test=0.537) total time=   0.0s
[CV 5/5; 37/120] START max_iter=20, penalty=none, solver=newton-cg..............
[CV 5/5; 37/120] END max_iter=20, penalty=none, solver=newton-cg; accuracy: (test=0.766) f1: (test=0.586) precision: (test=0.695) recall: (test=0.506) total time=   0.0s
[CV 1/5; 38/120] START max_iter=20, penalty=none, solver=liblinear..............
[CV 1/5; 38/120] END max_iter=20, penalty=none, solver=liblinear; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=   0.0s
[CV 2/5; 38/120] START max_iter=20, penalty=none, solver=liblinear..............
[CV 

...

[CV 4/5; 76/120] END max_iter=100, penalty=none, solver=lbfgs; accuracy: (test=0.719) f1: (test=0.557) precision: (test=0.579) recall: (test=0.537) total time=   0.0s
[CV 5/5; 76/120] START max_iter=100, penalty=none, solver=lbfgs.................
[CV 5/5; 76/120] END max_iter=100, penalty=none, solver=lbfgs; accuracy: (test=0.766) f1: (test=0.586) precision: (test=0.695) recall: (test=0.506) total time=   0.0s
[CV 1/5; 77/120] START max_iter=100, penalty=none, solver=newton-cg.............
[CV 1/5; 77/120] END max_iter=100, penalty=none, solver=newton-cg; accuracy: (test=0.739) f1: (test=0.552) precision: (test=0.625) recall: (test=0.494) total time=   0.1s
[CV 2/5; 77/120] START max_iter=100, penalty=none, solver=newton-cg.............
[CV 2/5; 77/120] END max_iter=100, penalty=none, solver=newton-cg; accuracy: (test=0.791) f1: (test=0.574) precision: (test=0.875) recall: (test=0.427) total time=   0.0s
[CV 3/5; 77/120] START max_iter=100, penalty=none, solver=newton-cg.............


[CV 2/5; 85/120] END max_iter=200, penalty=l1, solver=saga; accuracy: (test=0.767) f1: (test=0.517) precision: (test=0.816) recall: (test=0.378) total time=   0.0s
[CV 3/5; 85/120] START max_iter=200, penalty=l1, solver=saga....................
[CV 3/5; 85/120] END max_iter=200, penalty=l1, solver=saga; accuracy: (test=0.715) f1: (test=0.482) precision: (test=0.600) recall: (test=0.402) total time=   0.0s
[CV 4/5; 85/120] START max_iter=200, penalty=l1, solver=saga....................
[CV 4/5; 85/120] END max_iter=200, penalty=l1, solver=saga; accuracy: (test=0.751) f1: (test=0.569) precision: (test=0.661) recall: (test=0.500) total time=   0.0s
[CV 5/5; 85/120] START max_iter=200, penalty=l1, solver=saga....................
[CV 5/5; 85/120] END max_iter=200, penalty=l1, solver=saga; accuracy: (test=0.762) f1: (test=0.535) precision: (test=0.739) recall: (test=0.420) total time=   0.0s
[CV 1/5; 86/120] START max_iter=200, penalty=l2, solver=lbfgs...................
[CV 1/5; 86/120] END

[CV 1/5; 96/120] END max_iter=200, penalty=none, solver=lbfgs; accuracy: (test=0.739) f1: (test=0.552) precision: (test=0.625) recall: (test=0.494) total time=   0.1s
[CV 2/5; 96/120] START max_iter=200, penalty=none, solver=lbfgs.................
[CV 2/5; 96/120] END max_iter=200, penalty=none, solver=lbfgs; accuracy: (test=0.791) f1: (test=0.574) precision: (test=0.875) recall: (test=0.427) total time=   0.1s
[CV 3/5; 96/120] START max_iter=200, penalty=none, solver=lbfgs.................
[CV 3/5; 96/120] END max_iter=200, penalty=none, solver=lbfgs; accuracy: (test=0.711) f1: (test=0.520) precision: (test=0.574) recall: (test=0.476) total time=   0.1s
[CV 4/5; 96/120] START max_iter=200, penalty=none, solver=lbfgs.................
[CV 4/5; 96/120] END max_iter=200, penalty=none, solver=lbfgs; accuracy: (test=0.719) f1: (test=0.557) precision: (test=0.579) recall: (test=0.537) total time=   0.0s
[CV 5/5; 96/120] START max_iter=200, penalty=none, solver=lbfgs.................
[CV 5/5;

[CV 2/5; 106/120] END max_iter=500, penalty=l2, solver=lbfgs; accuracy: (test=0.763) f1: (test=0.504) precision: (test=0.811) recall: (test=0.366) total time=   0.0s
[CV 3/5; 106/120] START max_iter=500, penalty=l2, solver=lbfgs..................
[CV 3/5; 106/120] END max_iter=500, penalty=l2, solver=lbfgs; accuracy: (test=0.711) f1: (test=0.478) precision: (test=0.589) recall: (test=0.402) total time=   0.0s
[CV 4/5; 106/120] START max_iter=500, penalty=l2, solver=lbfgs..................
[CV 4/5; 106/120] END max_iter=500, penalty=l2, solver=lbfgs; accuracy: (test=0.751) f1: (test=0.575) precision: (test=0.656) recall: (test=0.512) total time=   0.0s
[CV 5/5; 106/120] START max_iter=500, penalty=l2, solver=lbfgs..................
[CV 5/5; 106/120] END max_iter=500, penalty=l2, solver=lbfgs; accuracy: (test=0.778) f1: (test=0.574) precision: (test=0.771) recall: (test=0.457) total time=   0.0s
[CV 1/5; 107/120] START max_iter=500, penalty=l2, solver=newton-cg..............
[CV 1/5; 107

[CV 3/5; 114/120] END max_iter=500, penalty=elasticnet, solver=sag; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=   0.0s
[CV 4/5; 114/120] START max_iter=500, penalty=elasticnet, solver=sag............
[CV 4/5; 114/120] END max_iter=500, penalty=elasticnet, solver=sag; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=   0.0s
[CV 5/5; 114/120] START max_iter=500, penalty=elasticnet, solver=sag............
[CV 5/5; 114/120] END max_iter=500, penalty=elasticnet, solver=sag; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=   0.0s
[CV 1/5; 115/120] START max_iter=500, penalty=elasticnet, solver=saga...........
[CV 1/5; 115/120] END max_iter=500, penalty=elasticnet, solver=saga; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=   0.0s
[CV 2/5; 115/120] START max_iter=500, penalty=elasticnet, solver=saga...........
[CV 2/5; 115/120] E

In [9]:
# Build the model with best parameters and test
print(f"----- GRID SEARCH CV FOR LOGISTIC REGRESSION -----")
print(f"Best Parameters {lg_grid.best_params}")
print(f"F1 Score {lg_grid.f1}")
print(f"Precision Score {lg_grid.precision}")
print(f"Recall Score {lg_grid.recall}")
print(f"Accuracy Score {lg_grid.accuracy}")

----- GRID SEARCH CV FOR LOGISTIC REGRESSION -----
Best Parameters {'max_iter': 10, 'penalty': 'none', 'solver': 'lbfgs'}
F1 Score 0.5624491831105016
Precision Score 0.6804768118822719
Recall Score 0.4853959650707618
Accuracy Score 0.7524193548387097


### 3.2 SVM 

In [10]:
# search CV parameters for SVM
svm_param_grid = {
    'kernel': ["linear", "poly", "rbf", "sigmoid"],
    'degree': [2, 3, 4, 5],
    'max_iter': [10, 20, 50, 100, 200, 500]
}


#### 3.2.1 Randomised Search CV - SVM

In [11]:
# initialize the SVM Model
svm_rand = RandomAndGridSearchCV(X_train=X_train_promapen,
                              y_train=y_train_promapen,
                              X_test=X_test_promapen,
                              y_test=y_test_promapen,
                              base_model_id=0,
                              classifier=SVC,
                              params=svm_param_grid)

# estimate the best parameters for SVM
svm_rand.estimate_best_params()

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5; 1/10] START degree=3, kernel=linear, max_iter=20.......................
[CV 1/5; 1/10] END degree=3, kernel=linear, max_iter=20; accuracy: (test=0.406) f1: (test=0.471) precision: (test=0.332) recall: (test=0.815) total time=   0.0s
[CV 2/5; 1/10] START degree=3, kernel=linear, max_iter=20.......................
[CV 2/5; 1/10] END degree=3, kernel=linear, max_iter=20; accuracy: (test=0.313) f1: (test=0.477) precision: (test=0.318) recall: (test=0.951) total time=   0.0s
[CV 3/5; 1/10] START degree=3, kernel=linear, max_iter=20.......................
[CV 3/5; 1/10] END degree=3, kernel=linear, max_iter=20; accuracy: (test=0.494) f1: (test=0.330) precision: (test=0.292) recall: (test=0.378) total time=   0.0s
[CV 4/5; 1/10] START degree=3, kernel=linear, max_iter=20.......................
[CV 4/5; 1/10] END degree=3, kernel=linear, max_iter=20; accuracy: (test=0.321) f1: (test=0.486) precision: (test=0.324) recall: (te

[CV 5/5; 7/10] END degree=4, kernel=linear, max_iter=50; accuracy: (test=0.298) f1: (test=0.420) precision: (test=0.288) recall: (test=0.778) total time=   0.0s
[CV 1/5; 8/10] START degree=2, kernel=rbf, max_iter=200.........................
[CV 1/5; 8/10] END degree=2, kernel=rbf, max_iter=200; accuracy: (test=0.719) f1: (test=0.602) precision: (test=0.558) recall: (test=0.654) total time=   0.0s
[CV 2/5; 8/10] START degree=2, kernel=rbf, max_iter=200.........................
[CV 2/5; 8/10] END degree=2, kernel=rbf, max_iter=200; accuracy: (test=0.727) f1: (test=0.521) precision: (test=0.617) recall: (test=0.451) total time=   0.0s
[CV 3/5; 8/10] START degree=2, kernel=rbf, max_iter=200.........................
[CV 3/5; 8/10] END degree=2, kernel=rbf, max_iter=200; accuracy: (test=0.711) f1: (test=0.507) precision: (test=0.578) recall: (test=0.451) total time=   0.0s
[CV 4/5; 8/10] START degree=2, kernel=rbf, max_iter=200.........................
[CV 4/5; 8/10] END degree=2, kernel=rb

In [12]:
# Build the model with best parameters and test
print(f"----- RANDOMIZED SEARCH CV FOR SVM -----")
print(f"Best Parameters {svm_rand.best_params}")
print(f"F1 Score {svm_rand.f1}")
print(f"Precision Score {svm_rand.precision}")
print(f"Recall Score {svm_rand.recall}")
print(f"Accuracy Score {svm_rand.accuracy}")

----- RANDOMIZED SEARCH CV FOR SVM -----
Best Parameters {'max_iter': 200, 'kernel': 'rbf', 'degree': 2}
F1 Score 0.5435638848747055
Precision Score 0.5786550327088908
Recall Score 0.5246010237880157
Accuracy Score 0.7162391501489831


#### 3.2.2 Grid Search CV - SVM

In [13]:
svm_grid = RandomAndGridSearchCV(X_train=X_train_promapen,
                                 y_train=y_train_promapen,
                                 X_test=X_test_promapen,
                                 y_test=y_test_promapen,
                                 base_model_id=1,
                                 classifier=SVC,
                                 params=svm_param_grid)

svm_grid.estimate_best_params()

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5; 1/96] START degree=2, kernel=linear, max_iter=10.......................
[CV 1/5; 1/96] END degree=2, kernel=linear, max_iter=10; accuracy: (test=0.305) f1: (test=0.433) precision: (test=0.295) recall: (test=0.815) total time=   0.0s
[CV 2/5; 1/96] START degree=2, kernel=linear, max_iter=10.......................
[CV 2/5; 1/96] END degree=2, kernel=linear, max_iter=10; accuracy: (test=0.261) f1: (test=0.391) precision: (test=0.268) recall: (test=0.720) total time=   0.0s
[CV 3/5; 1/96] START degree=2, kernel=linear, max_iter=10.......................
[CV 3/5; 1/96] END degree=2, kernel=linear, max_iter=10; accuracy: (test=0.297) f1: (test=0.407) precision: (test=0.282) recall: (test=0.732) total time=   0.0s
[CV 4/5; 1/96] START degree=2, kernel=linear, max_iter=10.......................
[CV 4/5; 1/96] END degree=2, kernel=linear, max_iter=10; accuracy: (test=0.454) f1: (test=0.541) precision: (test=0.374) recall: (t

[CV 5/5; 8/96] END degree=2, kernel=poly, max_iter=20; accuracy: (test=0.306) f1: (test=0.445) precision: (test=0.301) recall: (test=0.852) total time=   0.0s
[CV 1/5; 9/96] START degree=2, kernel=poly, max_iter=50.........................
[CV 1/5; 9/96] END degree=2, kernel=poly, max_iter=50; accuracy: (test=0.687) f1: (test=0.435) precision: (test=0.526) recall: (test=0.370) total time=   0.0s
[CV 2/5; 9/96] START degree=2, kernel=poly, max_iter=50.........................
[CV 2/5; 9/96] END degree=2, kernel=poly, max_iter=50; accuracy: (test=0.462) f1: (test=0.518) precision: (test=0.367) recall: (test=0.878) total time=   0.0s
[CV 3/5; 9/96] START degree=2, kernel=poly, max_iter=50.........................
[CV 3/5; 9/96] END degree=2, kernel=poly, max_iter=50; accuracy: (test=0.333) f1: (test=0.458) precision: (test=0.312) recall: (test=0.854) total time=   0.0s
[CV 4/5; 9/96] START degree=2, kernel=poly, max_iter=50.........................
[CV 4/5; 9/96] END degree=2, kernel=poly

[CV 4/5; 16/96] END degree=2, kernel=rbf, max_iter=100; accuracy: (test=0.566) f1: (test=0.509) precision: (test=0.406) recall: (test=0.683) total time=   0.0s
[CV 5/5; 16/96] START degree=2, kernel=rbf, max_iter=100........................
[CV 5/5; 16/96] END degree=2, kernel=rbf, max_iter=100; accuracy: (test=0.387) f1: (test=0.377) precision: (test=0.282) recall: (test=0.568) total time=   0.0s
[CV 1/5; 17/96] START degree=2, kernel=rbf, max_iter=200........................
[CV 1/5; 17/96] END degree=2, kernel=rbf, max_iter=200; accuracy: (test=0.719) f1: (test=0.602) precision: (test=0.558) recall: (test=0.654) total time=   0.0s
[CV 2/5; 17/96] START degree=2, kernel=rbf, max_iter=200........................
[CV 2/5; 17/96] END degree=2, kernel=rbf, max_iter=200; accuracy: (test=0.727) f1: (test=0.521) precision: (test=0.617) recall: (test=0.451) total time=   0.0s
[CV 3/5; 17/96] START degree=2, kernel=rbf, max_iter=200........................
[CV 3/5; 17/96] END degree=2, kernel

[CV 3/5; 23/96] END degree=2, kernel=sigmoid, max_iter=200; accuracy: (test=0.426) f1: (test=0.302) precision: (test=0.252) recall: (test=0.378) total time=   0.0s
[CV 4/5; 23/96] START degree=2, kernel=sigmoid, max_iter=200....................
[CV 4/5; 23/96] END degree=2, kernel=sigmoid, max_iter=200; accuracy: (test=0.382) f1: (test=0.312) precision: (test=0.246) recall: (test=0.427) total time=   0.0s
[CV 5/5; 23/96] START degree=2, kernel=sigmoid, max_iter=200....................
[CV 5/5; 23/96] END degree=2, kernel=sigmoid, max_iter=200; accuracy: (test=0.319) f1: (test=0.235) precision: (test=0.186) recall: (test=0.321) total time=   0.0s
[CV 1/5; 24/96] START degree=2, kernel=sigmoid, max_iter=500....................
[CV 1/5; 24/96] END degree=2, kernel=sigmoid, max_iter=500; accuracy: (test=0.494) f1: (test=0.113) precision: (test=0.131) recall: (test=0.099) total time=   0.0s
[CV 2/5; 24/96] START degree=2, kernel=sigmoid, max_iter=500....................
[CV 2/5; 24/96] END 

[CV 5/5; 32/96] END degree=3, kernel=poly, max_iter=20; accuracy: (test=0.278) f1: (test=0.435) precision: (test=0.292) recall: (test=0.852) total time=   0.0s
[CV 1/5; 33/96] START degree=3, kernel=poly, max_iter=50........................
[CV 1/5; 33/96] END degree=3, kernel=poly, max_iter=50; accuracy: (test=0.277) f1: (test=0.419) precision: (test=0.284) recall: (test=0.802) total time=   0.0s
[CV 2/5; 33/96] START degree=3, kernel=poly, max_iter=50........................
[CV 2/5; 33/96] END degree=3, kernel=poly, max_iter=50; accuracy: (test=0.293) f1: (test=0.443) precision: (test=0.299) recall: (test=0.854) total time=   0.0s
[CV 3/5; 33/96] START degree=3, kernel=poly, max_iter=50........................
[CV 3/5; 33/96] END degree=3, kernel=poly, max_iter=50; accuracy: (test=0.317) f1: (test=0.437) precision: (test=0.300) recall: (test=0.805) total time=   0.0s
[CV 4/5; 33/96] START degree=3, kernel=poly, max_iter=50........................
[CV 4/5; 33/96] END degree=3, kernel

...

[CV 1/5; 62/96] END degree=4, kernel=rbf, max_iter=20; accuracy: (test=0.518) f1: (test=0.326) precision: (test=0.299) recall: (test=0.358) total time=   0.0s
[CV 2/5; 62/96] START degree=4, kernel=rbf, max_iter=20.........................
[CV 2/5; 62/96] END degree=4, kernel=rbf, max_iter=20; accuracy: (test=0.618) f1: (test=0.451) precision: (test=0.429) recall: (test=0.476) total time=   0.0s
[CV 3/5; 62/96] START degree=4, kernel=rbf, max_iter=20.........................
[CV 3/5; 62/96] END degree=4, kernel=rbf, max_iter=20; accuracy: (test=0.622) f1: (test=0.277) precision: (test=0.375) recall: (test=0.220) total time=   0.0s
[CV 4/5; 62/96] START degree=4, kernel=rbf, max_iter=20.........................
[CV 4/5; 62/96] END degree=4, kernel=rbf, max_iter=20; accuracy: (test=0.430) f1: (test=0.174) precision: (test=0.167) recall: (test=0.183) total time=   0.0s
[CV 5/5; 62/96] START degree=4, kernel=rbf, max_iter=20.........................
[CV 5/5; 62/96] END degree=4, kernel=rbf

[CV 1/5; 70/96] END degree=4, kernel=sigmoid, max_iter=100; accuracy: (test=0.277) f1: (test=0.323) precision: (test=0.232) recall: (test=0.531) total time=   0.0s
[CV 2/5; 70/96] START degree=4, kernel=sigmoid, max_iter=100....................
[CV 2/5; 70/96] END degree=4, kernel=sigmoid, max_iter=100; accuracy: (test=0.305) f1: (test=0.397) precision: (test=0.278) recall: (test=0.695) total time=   0.0s
[CV 3/5; 70/96] START degree=4, kernel=sigmoid, max_iter=100....................
[CV 3/5; 70/96] END degree=4, kernel=sigmoid, max_iter=100; accuracy: (test=0.333) f1: (test=0.411) precision: (test=0.290) recall: (test=0.707) total time=   0.0s
[CV 4/5; 70/96] START degree=4, kernel=sigmoid, max_iter=100....................
[CV 4/5; 70/96] END degree=4, kernel=sigmoid, max_iter=100; accuracy: (test=0.361) f1: (test=0.323) precision: (test=0.248) recall: (test=0.463) total time=   0.0s
[CV 5/5; 70/96] START degree=4, kernel=sigmoid, max_iter=100....................
[CV 5/5; 70/96] END 

[CV 3/5; 77/96] END degree=5, kernel=linear, max_iter=200; accuracy: (test=0.691) f1: (test=0.528) precision: (test=0.531) recall: (test=0.524) total time=   0.0s
[CV 4/5; 77/96] START degree=5, kernel=linear, max_iter=200.....................
[CV 4/5; 77/96] END degree=5, kernel=linear, max_iter=200; accuracy: (test=0.703) f1: (test=0.526) precision: (test=0.554) recall: (test=0.500) total time=   0.0s
[CV 5/5; 77/96] START degree=5, kernel=linear, max_iter=200.....................
[CV 5/5; 77/96] END degree=5, kernel=linear, max_iter=200; accuracy: (test=0.540) f1: (test=0.477) precision: (test=0.380) recall: (test=0.642) total time=   0.0s
[CV 1/5; 78/96] START degree=5, kernel=linear, max_iter=500.....................
[CV 1/5; 78/96] END degree=5, kernel=linear, max_iter=500; accuracy: (test=0.751) f1: (test=0.530) precision: (test=0.686) recall: (test=0.432) total time=   0.0s
[CV 2/5; 78/96] START degree=5, kernel=linear, max_iter=500.....................
[CV 2/5; 78/96] END degr

[CV 2/5; 88/96] END degree=5, kernel=rbf, max_iter=100; accuracy: (test=0.651) f1: (test=0.549) precision: (test=0.477) recall: (test=0.646) total time=   0.0s
[CV 3/5; 88/96] START degree=5, kernel=rbf, max_iter=100........................
[CV 3/5; 88/96] END degree=5, kernel=rbf, max_iter=100; accuracy: (test=0.466) f1: (test=0.332) precision: (test=0.282) recall: (test=0.402) total time=   0.0s
[CV 4/5; 88/96] START degree=5, kernel=rbf, max_iter=100........................
[CV 4/5; 88/96] END degree=5, kernel=rbf, max_iter=100; accuracy: (test=0.566) f1: (test=0.509) precision: (test=0.406) recall: (test=0.683) total time=   0.0s
[CV 5/5; 88/96] START degree=5, kernel=rbf, max_iter=100........................
[CV 5/5; 88/96] END degree=5, kernel=rbf, max_iter=100; accuracy: (test=0.387) f1: (test=0.377) precision: (test=0.282) recall: (test=0.568) total time=   0.0s
[CV 1/5; 89/96] START degree=5, kernel=rbf, max_iter=200........................
[CV 1/5; 89/96] END degree=5, kernel

[CV 1/5; 95/96] END degree=5, kernel=sigmoid, max_iter=200; accuracy: (test=0.394) f1: (test=0.226) precision: (test=0.193) recall: (test=0.272) total time=   0.0s
[CV 2/5; 95/96] START degree=5, kernel=sigmoid, max_iter=200....................
[CV 2/5; 95/96] END degree=5, kernel=sigmoid, max_iter=200; accuracy: (test=0.418) f1: (test=0.367) precision: (test=0.286) recall: (test=0.512) total time=   0.0s
[CV 3/5; 95/96] START degree=5, kernel=sigmoid, max_iter=200....................
[CV 3/5; 95/96] END degree=5, kernel=sigmoid, max_iter=200; accuracy: (test=0.426) f1: (test=0.302) precision: (test=0.252) recall: (test=0.378) total time=   0.0s
[CV 4/5; 95/96] START degree=5, kernel=sigmoid, max_iter=200....................
[CV 4/5; 95/96] END degree=5, kernel=sigmoid, max_iter=200; accuracy: (test=0.382) f1: (test=0.312) precision: (test=0.246) recall: (test=0.427) total time=   0.0s
[CV 5/5; 95/96] START degree=5, kernel=sigmoid, max_iter=200....................
[CV 5/5; 95/96] END 

In [14]:
# Build the model with best parameters and test
print(f"----- GRID SEARCH CV FOR SVM -----")
print(f"Best Parameters {svm_grid.best_params}")
print(f"F1 Score {svm_grid.f1}")
print(f"Precision Score {svm_grid.precision}")
print(f"Recall Score {svm_grid.recall}")
print(f"Accuracy Score {svm_grid.accuracy}")

----- GRID SEARCH CV FOR SVM -----
Best Parameters {'degree': 2, 'kernel': 'rbf', 'max_iter': 200}
F1 Score 0.5435638848747055
Precision Score 0.5786550327088908
Recall Score 0.5246010237880157
Accuracy Score 0.7162391501489831


### 3.3 Decision Tree

In [15]:
# search CV parameters for decision tree
dt_param_grid = {
    'criterion': ["gini", "entropy"],
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [2, 5, 10, 15, 20]
}

#### 3.3.1 Randomised Search CV - Decision Tree

In [16]:
# initialize the Decision Tree Model
dt_rand = RandomAndGridSearchCV(X_train=X_train_promapen,
                              y_train=y_train_promapen,
                              X_test=X_test_promapen,
                              y_test=y_test_promapen,
                              base_model_id=0,
                              classifier=DecisionTreeClassifier,
                              params=dt_param_grid)

# estimate the best parameters for Decision Tree
dt_rand.estimate_best_params()

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5; 1/10] START criterion=entropy, max_depth=5, min_samples_split=15.......
[CV 1/5; 1/10] END criterion=entropy, max_depth=5, min_samples_split=15; accuracy: (test=0.727) f1: (test=0.564) precision: (test=0.587) recall: (test=0.543) total time=   0.0s
[CV 2/5; 1/10] START criterion=entropy, max_depth=5, min_samples_split=15.......
[CV 2/5; 1/10] END criterion=entropy, max_depth=5, min_samples_split=15; accuracy: (test=0.727) f1: (test=0.443) precision: (test=0.675) recall: (test=0.329) total time=   0.0s
[CV 3/5; 1/10] START criterion=entropy, max_depth=5, min_samples_split=15.......
[CV 3/5; 1/10] END criterion=entropy, max_depth=5, min_samples_split=15; accuracy: (test=0.663) f1: (test=0.276) precision: (test=0.471) recall: (test=0.195) total time=   0.0s
[CV 4/5; 1/10] START criterion=entropy, max_depth=5, min_samples_split=15.......
[CV 4/5; 1/10] END criterion=entropy, max_depth=5, min_samples_split=15; accuracy: (

[CV 3/5; 7/10] END criterion=gini, max_depth=5, min_samples_split=5; accuracy: (test=0.687) f1: (test=0.426) precision: (test=0.537) recall: (test=0.354) total time=   0.0s
[CV 4/5; 7/10] START criterion=gini, max_depth=5, min_samples_split=5...........
[CV 4/5; 7/10] END criterion=gini, max_depth=5, min_samples_split=5; accuracy: (test=0.679) f1: (test=0.459) precision: (test=0.515) recall: (test=0.415) total time=   0.0s
[CV 5/5; 7/10] START criterion=gini, max_depth=5, min_samples_split=5...........
[CV 5/5; 7/10] END criterion=gini, max_depth=5, min_samples_split=5; accuracy: (test=0.766) f1: (test=0.547) precision: (test=0.745) recall: (test=0.432) total time=   0.0s
[CV 1/5; 8/10] START criterion=entropy, max_depth=10, min_samples_split=15......
[CV 1/5; 8/10] END criterion=entropy, max_depth=10, min_samples_split=15; accuracy: (test=0.651) f1: (test=0.514) precision: (test=0.469) recall: (test=0.568) total time=   0.0s
[CV 2/5; 8/10] START criterion=entropy, max_depth=10, min_sa

In [17]:
# Build the model with best parameters and test
print(f"----- RANDOMIZED SEARCH CV FOR Decision Tree -----")
print(f"Best Parameters {dt_rand.best_params}")
print(f"F1 Score {dt_rand.f1}")
print(f"Precision Score {dt_rand.precision}")
print(f"Recall Score {dt_rand.recall}")
print(f"Accuracy Score {dt_rand.accuracy}")

----- RANDOMIZED SEARCH CV FOR Decision Tree -----
Best Parameters {'min_samples_split': 15, 'max_depth': 5, 'criterion': 'gini'}
F1 Score 0.5043699226176128
Precision Score 0.6067160135462022
Recall Score 0.4366154772658838
Accuracy Score 0.7202973183054799


#### 3.3.2 Grid Search CV - Decision Tree 

In [18]:
dt_grid = RandomAndGridSearchCV(X_train=X_train_promapen,
                              y_train=y_train_promapen,
                              X_test=X_test_promapen,
                              y_test=y_test_promapen,
                              base_model_id=1,
                              classifier=DecisionTreeClassifier,
                              params=dt_param_grid)

dt_grid.estimate_best_params()

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV 1/5; 1/40] START criterion=gini, max_depth=5, min_samples_split=2...........
[CV 1/5; 1/40] END criterion=gini, max_depth=5, min_samples_split=2; accuracy: (test=0.743) f1: (test=0.590) precision: (test=0.613) recall: (test=0.568) total time=   0.0s
[CV 2/5; 1/40] START criterion=gini, max_depth=5, min_samples_split=2...........
[CV 2/5; 1/40] END criterion=gini, max_depth=5, min_samples_split=2; accuracy: (test=0.711) f1: (test=0.463) precision: (test=0.596) recall: (test=0.378) total time=   0.0s
[CV 3/5; 1/40] START criterion=gini, max_depth=5, min_samples_split=2...........
[CV 3/5; 1/40] END criterion=gini, max_depth=5, min_samples_split=2; accuracy: (test=0.687) f1: (test=0.426) precision: (test=0.537) recall: (test=0.354) total time=   0.0s
[CV 4/5; 1/40] START criterion=gini, max_depth=5, min_samples_split=2...........
[CV 4/5; 1/40] END criterion=gini, max_depth=5, min_samples_split=2; accuracy: (test=0.679) f1:

[CV 1/5; 9/40] END criterion=gini, max_depth=10, min_samples_split=15; accuracy: (test=0.671) f1: (test=0.453) precision: (test=0.493) recall: (test=0.420) total time=   0.0s
[CV 2/5; 9/40] START criterion=gini, max_depth=10, min_samples_split=15.........
[CV 2/5; 9/40] END criterion=gini, max_depth=10, min_samples_split=15; accuracy: (test=0.723) f1: (test=0.517) precision: (test=0.607) recall: (test=0.451) total time=   0.0s
[CV 3/5; 9/40] START criterion=gini, max_depth=10, min_samples_split=15.........
[CV 3/5; 9/40] END criterion=gini, max_depth=10, min_samples_split=15; accuracy: (test=0.659) f1: (test=0.437) precision: (test=0.478) recall: (test=0.402) total time=   0.0s
[CV 4/5; 9/40] START criterion=gini, max_depth=10, min_samples_split=15.........
[CV 4/5; 9/40] END criterion=gini, max_depth=10, min_samples_split=15; accuracy: (test=0.627) f1: (test=0.436) precision: (test=0.434) recall: (test=0.439) total time=   0.0s
[CV 5/5; 9/40] START criterion=gini, max_depth=10, min_sa

[CV 3/5; 15/40] END criterion=gini, max_depth=15, min_samples_split=20; accuracy: (test=0.627) f1: (test=0.450) precision: (test=0.437) recall: (test=0.463) total time=   0.0s
[CV 4/5; 15/40] START criterion=gini, max_depth=15, min_samples_split=20........
[CV 4/5; 15/40] END criterion=gini, max_depth=15, min_samples_split=20; accuracy: (test=0.631) f1: (test=0.459) precision: (test=0.443) recall: (test=0.476) total time=   0.0s
[CV 5/5; 15/40] START criterion=gini, max_depth=15, min_samples_split=20........
[CV 5/5; 15/40] END criterion=gini, max_depth=15, min_samples_split=20; accuracy: (test=0.702) f1: (test=0.532) precision: (test=0.545) recall: (test=0.519) total time=   0.0s
[CV 1/5; 16/40] START criterion=gini, max_depth=20, min_samples_split=2.........
[CV 1/5; 16/40] END criterion=gini, max_depth=20, min_samples_split=2; accuracy: (test=0.667) f1: (test=0.515) precision: (test=0.489) recall: (test=0.543) total time=   0.0s
[CV 2/5; 16/40] START criterion=gini, max_depth=20, mi

[CV 4/5; 22/40] END criterion=entropy, max_depth=5, min_samples_split=5; accuracy: (test=0.675) f1: (test=0.526) precision: (test=0.506) recall: (test=0.549) total time=   0.0s
[CV 5/5; 22/40] START criterion=entropy, max_depth=5, min_samples_split=5.......
[CV 5/5; 22/40] END criterion=entropy, max_depth=5, min_samples_split=5; accuracy: (test=0.722) f1: (test=0.531) precision: (test=0.591) recall: (test=0.481) total time=   0.0s
[CV 1/5; 23/40] START criterion=entropy, max_depth=5, min_samples_split=10......
[CV 1/5; 23/40] END criterion=entropy, max_depth=5, min_samples_split=10; accuracy: (test=0.731) f1: (test=0.568) precision: (test=0.595) recall: (test=0.543) total time=   0.0s
[CV 2/5; 23/40] START criterion=entropy, max_depth=5, min_samples_split=10......
[CV 2/5; 23/40] END criterion=entropy, max_depth=5, min_samples_split=10; accuracy: (test=0.739) f1: (test=0.480) precision: (test=0.698) recall: (test=0.366) total time=   0.0s
[CV 3/5; 23/40] START criterion=entropy, max_de

[CV 1/5; 29/40] END criterion=entropy, max_depth=10, min_samples_split=15; accuracy: (test=0.651) f1: (test=0.514) precision: (test=0.469) recall: (test=0.568) total time=   0.0s
[CV 2/5; 29/40] START criterion=entropy, max_depth=10, min_samples_split=15.....
[CV 2/5; 29/40] END criterion=entropy, max_depth=10, min_samples_split=15; accuracy: (test=0.727) f1: (test=0.528) precision: (test=0.613) recall: (test=0.463) total time=   0.0s
[CV 3/5; 29/40] START criterion=entropy, max_depth=10, min_samples_split=15.....
[CV 3/5; 29/40] END criterion=entropy, max_depth=10, min_samples_split=15; accuracy: (test=0.643) f1: (test=0.395) precision: (test=0.446) recall: (test=0.354) total time=   0.0s
[CV 4/5; 29/40] START criterion=entropy, max_depth=10, min_samples_split=15.....
[CV 4/5; 29/40] END criterion=entropy, max_depth=10, min_samples_split=15; accuracy: (test=0.655) f1: (test=0.419) precision: (test=0.470) recall: (test=0.378) total time=   0.0s
[CV 5/5; 29/40] START criterion=entropy, 

[CV 4/5; 35/40] END criterion=entropy, max_depth=15, min_samples_split=20; accuracy: (test=0.651) f1: (test=0.453) precision: (test=0.468) recall: (test=0.439) total time=   0.0s
[CV 5/5; 35/40] START criterion=entropy, max_depth=15, min_samples_split=20.....
[CV 5/5; 35/40] END criterion=entropy, max_depth=15, min_samples_split=20; accuracy: (test=0.673) f1: (test=0.477) precision: (test=0.500) recall: (test=0.457) total time=   0.0s
[CV 1/5; 36/40] START criterion=entropy, max_depth=20, min_samples_split=2......
[CV 1/5; 36/40] END criterion=entropy, max_depth=20, min_samples_split=2; accuracy: (test=0.663) f1: (test=0.548) precision: (test=0.486) recall: (test=0.630) total time=   0.0s
[CV 2/5; 36/40] START criterion=entropy, max_depth=20, min_samples_split=2......
[CV 2/5; 36/40] END criterion=entropy, max_depth=20, min_samples_split=2; accuracy: (test=0.703) f1: (test=0.565) precision: (test=0.545) recall: (test=0.585) total time=   0.0s
[CV 3/5; 36/40] START criterion=entropy, ma

In [19]:
# Build the model with best parameters and test
print(f"----- GRID SEARCH CV FOR DECISION TREE -----")
print(f"Best Parameters {dt_grid.best_params}")
print(f"F1 Score {dt_grid.f1}")
print(f"Precision Score {dt_grid.precision}")
print(f"Recall Score {dt_grid.recall}")
print(f"Accuracy Score {dt_grid.accuracy}")

----- GRID SEARCH CV FOR DECISION TREE -----
Best Parameters {'criterion': 'entropy', 'max_depth': 20, 'min_samples_split': 2}
F1 Score 0.507570627320286
Precision Score 0.49549094476060773
Recall Score 0.5246913580246914
Accuracy Score 0.6672334499287473


### 3.4 Random Forest

In [20]:
# search CV parameters for Random Forest 
rf_param_grid = {
    'n_estimators': [50, 100, 200, 500],
    'max_depth': [5, 10, 20, 50],
    'min_samples_split': [2, 5, 10, 20],
    'criterion': ["gini", "entropy"]
}


#### 3.4.1 Randomised Search CV - Random Forest

In [21]:
# initialize the Random Forest Model
rf_rand = RandomAndGridSearchCV(X_train=X_train_promapen,
                              y_train=y_train_promapen,
                              X_test=X_test_promapen,
                              y_test=y_test_promapen,
                              base_model_id=0,
                              classifier=RandomForestClassifier,
                              params=dt_param_grid)

# estimate the best parameters for Random Forest
rf_rand.estimate_best_params()

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5; 1/10] START criterion=entropy, max_depth=10, min_samples_split=20......
[CV 1/5; 1/10] END criterion=entropy, max_depth=10, min_samples_split=20; accuracy: (test=0.735) f1: (test=0.571) precision: (test=0.603) recall: (test=0.543) total time=   0.1s
[CV 2/5; 1/10] START criterion=entropy, max_depth=10, min_samples_split=20......
[CV 2/5; 1/10] END criterion=entropy, max_depth=10, min_samples_split=20; accuracy: (test=0.755) f1: (test=0.479) precision: (test=0.800) recall: (test=0.341) total time=   0.1s
[CV 3/5; 1/10] START criterion=entropy, max_depth=10, min_samples_split=20......
[CV 3/5; 1/10] END criterion=entropy, max_depth=10, min_samples_split=20; accuracy: (test=0.723) f1: (test=0.481) precision: (test=0.627) recall: (test=0.390) total time=   0.1s
[CV 4/5; 1/10] START criterion=entropy, max_depth=10, min_samples_split=20......
[CV 4/5; 1/10] END criterion=entropy, max_depth=10, min_samples_split=20; accurac

[CV 3/5; 7/10] END criterion=gini, max_depth=10, min_samples_split=5; accuracy: (test=0.707) f1: (test=0.475) precision: (test=0.579) recall: (test=0.402) total time=   0.1s
[CV 4/5; 7/10] START criterion=gini, max_depth=10, min_samples_split=5..........
[CV 4/5; 7/10] END criterion=gini, max_depth=10, min_samples_split=5; accuracy: (test=0.703) f1: (test=0.479) precision: (test=0.567) recall: (test=0.415) total time=   0.1s
[CV 5/5; 7/10] START criterion=gini, max_depth=10, min_samples_split=5..........
[CV 5/5; 7/10] END criterion=gini, max_depth=10, min_samples_split=5; accuracy: (test=0.774) f1: (test=0.588) precision: (test=0.727) recall: (test=0.494) total time=   0.1s
[CV 1/5; 8/10] START criterion=gini, max_depth=10, min_samples_split=2..........
[CV 1/5; 8/10] END criterion=gini, max_depth=10, min_samples_split=2; accuracy: (test=0.759) f1: (test=0.605) precision: (test=0.648) recall: (test=0.568) total time=   0.2s
[CV 2/5; 8/10] START criterion=gini, max_depth=10, min_sample

In [22]:
# Build the model with best parameters and test
print(f"----- RANDOMIZED SEARCH CV FOR RANDOM FOREST -----")
print(f"Best Parameters {rf_rand.best_params}")
print(f"F1 Score {rf_rand.f1}")
print(f"Precision Score {rf_rand.precision}")
print(f"Recall Score {rf_rand.recall}")
print(f"Accuracy Score {rf_rand.accuracy}")

----- RANDOMIZED SEARCH CV FOR RANDOM FOREST -----
Best Parameters {'min_samples_split': 15, 'max_depth': 15, 'criterion': 'gini'}
F1 Score 0.5446374507860272
Precision Score 0.6667150908946574
Recall Score 0.47094248720264986
Accuracy Score 0.7444034201321414


#### 3.4.2 Grid Search CV - Random Forest

In [23]:
rf_grid = RandomAndGridSearchCV(X_train=X_train_promapen,
                              y_train=y_train_promapen,
                              X_test=X_test_promapen,
                              y_test=y_test_promapen,
                              base_model_id=1,
                              classifier=RandomForestClassifier,
                              params=rf_param_grid)

rf_grid.estimate_best_params()

Fitting 5 folds for each of 128 candidates, totalling 640 fits
[CV 1/5; 1/128] START criterion=gini, max_depth=5, min_samples_split=2, n_estimators=50
[CV 1/5; 1/128] END criterion=gini, max_depth=5, min_samples_split=2, n_estimators=50; accuracy: (test=0.755) f1: (test=0.579) precision: (test=0.656) recall: (test=0.519) total time=   0.1s
[CV 2/5; 1/128] START criterion=gini, max_depth=5, min_samples_split=2, n_estimators=50
[CV 2/5; 1/128] END criterion=gini, max_depth=5, min_samples_split=2, n_estimators=50; accuracy: (test=0.731) f1: (test=0.407) precision: (test=0.742) recall: (test=0.280) total time=   0.1s
[CV 3/5; 1/128] START criterion=gini, max_depth=5, min_samples_split=2, n_estimators=50
[CV 3/5; 1/128] END criterion=gini, max_depth=5, min_samples_split=2, n_estimators=50; accuracy: (test=0.715) f1: (test=0.458) precision: (test=0.612) recall: (test=0.366) total time=   0.1s
[CV 4/5; 1/128] START criterion=gini, max_depth=5, min_samples_split=2, n_estimators=50
[CV 4/5; 1/1

[CV 1/5; 7/128] END criterion=gini, max_depth=5, min_samples_split=5, n_estimators=200; accuracy: (test=0.771) f1: (test=0.601) precision: (test=0.694) recall: (test=0.531) total time=   0.2s
[CV 2/5; 7/128] START criterion=gini, max_depth=5, min_samples_split=5, n_estimators=200
[CV 2/5; 7/128] END criterion=gini, max_depth=5, min_samples_split=5, n_estimators=200; accuracy: (test=0.731) f1: (test=0.396) precision: (test=0.759) recall: (test=0.268) total time=   0.2s
[CV 3/5; 7/128] START criterion=gini, max_depth=5, min_samples_split=5, n_estimators=200
[CV 3/5; 7/128] END criterion=gini, max_depth=5, min_samples_split=5, n_estimators=200; accuracy: (test=0.711) f1: (test=0.446) precision: (test=0.604) recall: (test=0.354) total time=   0.2s
[CV 4/5; 7/128] START criterion=gini, max_depth=5, min_samples_split=5, n_estimators=200
[CV 4/5; 7/128] END criterion=gini, max_depth=5, min_samples_split=5, n_estimators=200; accuracy: (test=0.735) f1: (test=0.529) precision: (test=0.638) recal

[CV 5/5; 12/128] END criterion=gini, max_depth=5, min_samples_split=10, n_estimators=500; accuracy: (test=0.762) f1: (test=0.520) precision: (test=0.762) recall: (test=0.395) total time=   0.5s
[CV 1/5; 13/128] START criterion=gini, max_depth=5, min_samples_split=20, n_estimators=50
[CV 1/5; 13/128] END criterion=gini, max_depth=5, min_samples_split=20, n_estimators=50; accuracy: (test=0.743) f1: (test=0.549) precision: (test=0.639) recall: (test=0.481) total time=   0.1s
[CV 2/5; 13/128] START criterion=gini, max_depth=5, min_samples_split=20, n_estimators=50
[CV 2/5; 13/128] END criterion=gini, max_depth=5, min_samples_split=20, n_estimators=50; accuracy: (test=0.743) f1: (test=0.429) precision: (test=0.800) recall: (test=0.293) total time=   0.1s
[CV 3/5; 13/128] START criterion=gini, max_depth=5, min_samples_split=20, n_estimators=50
[CV 3/5; 13/128] END criterion=gini, max_depth=5, min_samples_split=20, n_estimators=50; accuracy: (test=0.715) f1: (test=0.458) precision: (test=0.61

[CV 5/5; 18/128] END criterion=gini, max_depth=10, min_samples_split=2, n_estimators=100; accuracy: (test=0.770) f1: (test=0.558) precision: (test=0.750) recall: (test=0.444) total time=   0.1s
[CV 1/5; 19/128] START criterion=gini, max_depth=10, min_samples_split=2, n_estimators=200
[CV 1/5; 19/128] END criterion=gini, max_depth=10, min_samples_split=2, n_estimators=200; accuracy: (test=0.751) f1: (test=0.592) precision: (test=0.634) recall: (test=0.556) total time=   0.3s
[CV 2/5; 19/128] START criterion=gini, max_depth=10, min_samples_split=2, n_estimators=200
[CV 2/5; 19/128] END criterion=gini, max_depth=10, min_samples_split=2, n_estimators=200; accuracy: (test=0.755) f1: (test=0.470) precision: (test=0.818) recall: (test=0.329) total time=   0.3s
[CV 3/5; 19/128] START criterion=gini, max_depth=10, min_samples_split=2, n_estimators=200
[CV 3/5; 19/128] END criterion=gini, max_depth=10, min_samples_split=2, n_estimators=200; accuracy: (test=0.691) f1: (test=0.462) precision: (tes

[CV 4/5; 24/128] END criterion=gini, max_depth=10, min_samples_split=5, n_estimators=500; accuracy: (test=0.715) f1: (test=0.510) precision: (test=0.587) recall: (test=0.451) total time=   0.7s
[CV 5/5; 24/128] START criterion=gini, max_depth=10, min_samples_split=5, n_estimators=500
[CV 5/5; 24/128] END criterion=gini, max_depth=10, min_samples_split=5, n_estimators=500; accuracy: (test=0.778) f1: (test=0.574) precision: (test=0.771) recall: (test=0.457) total time=   0.7s
[CV 1/5; 25/128] START criterion=gini, max_depth=10, min_samples_split=10, n_estimators=50
[CV 1/5; 25/128] END criterion=gini, max_depth=10, min_samples_split=10, n_estimators=50; accuracy: (test=0.743) f1: (test=0.584) precision: (test=0.616) recall: (test=0.556) total time=   0.1s
[CV 2/5; 25/128] START criterion=gini, max_depth=10, min_samples_split=10, n_estimators=50
[CV 2/5; 25/128] END criterion=gini, max_depth=10, min_samples_split=10, n_estimators=50; accuracy: (test=0.763) f1: (test=0.512) precision: (tes

...

[CV 3/5; 101/128] END criterion=entropy, max_depth=20, min_samples_split=5, n_estimators=50; accuracy: (test=0.691) f1: (test=0.462) precision: (test=0.541) recall: (test=0.402) total time=   0.1s
[CV 4/5; 101/128] START criterion=entropy, max_depth=20, min_samples_split=5, n_estimators=50
[CV 4/5; 101/128] END criterion=entropy, max_depth=20, min_samples_split=5, n_estimators=50; accuracy: (test=0.719) f1: (test=0.521) precision: (test=0.594) recall: (test=0.463) total time=   0.1s
[CV 5/5; 101/128] START criterion=entropy, max_depth=20, min_samples_split=5, n_estimators=50
[CV 5/5; 101/128] END criterion=entropy, max_depth=20, min_samples_split=5, n_estimators=50; accuracy: (test=0.766) f1: (test=0.540) precision: (test=0.756) recall: (test=0.420) total time=   0.1s
[CV 1/5; 102/128] START criterion=entropy, max_depth=20, min_samples_split=5, n_estimators=100
[CV 1/5; 102/128] END criterion=entropy, max_depth=20, min_samples_split=5, n_estimators=100; accuracy: (test=0.767) f1: (test

[CV 1/5; 107/128] END criterion=entropy, max_depth=20, min_samples_split=10, n_estimators=200; accuracy: (test=0.755) f1: (test=0.611) precision: (test=0.632) recall: (test=0.593) total time=   0.3s
[CV 2/5; 107/128] START criterion=entropy, max_depth=20, min_samples_split=10, n_estimators=200
[CV 2/5; 107/128] END criterion=entropy, max_depth=20, min_samples_split=10, n_estimators=200; accuracy: (test=0.751) f1: (test=0.456) precision: (test=0.812) recall: (test=0.317) total time=   0.3s
[CV 3/5; 107/128] START criterion=entropy, max_depth=20, min_samples_split=10, n_estimators=200
[CV 3/5; 107/128] END criterion=entropy, max_depth=20, min_samples_split=10, n_estimators=200; accuracy: (test=0.727) f1: (test=0.500) precision: (test=0.630) recall: (test=0.415) total time=   0.3s
[CV 4/5; 107/128] START criterion=entropy, max_depth=20, min_samples_split=10, n_estimators=200
[CV 4/5; 107/128] END criterion=entropy, max_depth=20, min_samples_split=10, n_estimators=200; accuracy: (test=0.71

[CV 4/5; 112/128] END criterion=entropy, max_depth=20, min_samples_split=20, n_estimators=500; accuracy: (test=0.711) f1: (test=0.500) precision: (test=0.581) recall: (test=0.439) total time=   0.8s
[CV 5/5; 112/128] START criterion=entropy, max_depth=20, min_samples_split=20, n_estimators=500
[CV 5/5; 112/128] END criterion=entropy, max_depth=20, min_samples_split=20, n_estimators=500; accuracy: (test=0.782) f1: (test=0.585) precision: (test=0.776) recall: (test=0.469) total time=   0.8s
[CV 1/5; 113/128] START criterion=entropy, max_depth=50, min_samples_split=2, n_estimators=50
[CV 1/5; 113/128] END criterion=entropy, max_depth=50, min_samples_split=2, n_estimators=50; accuracy: (test=0.739) f1: (test=0.564) precision: (test=0.618) recall: (test=0.519) total time=   0.1s
[CV 2/5; 113/128] START criterion=entropy, max_depth=50, min_samples_split=2, n_estimators=50
[CV 2/5; 113/128] END criterion=entropy, max_depth=50, min_samples_split=2, n_estimators=50; accuracy: (test=0.755) f1: (

[CV 3/5; 118/128] END criterion=entropy, max_depth=50, min_samples_split=5, n_estimators=100; accuracy: (test=0.699) f1: (test=0.460) precision: (test=0.561) recall: (test=0.390) total time=   0.2s
[CV 4/5; 118/128] START criterion=entropy, max_depth=50, min_samples_split=5, n_estimators=100
[CV 4/5; 118/128] END criterion=entropy, max_depth=50, min_samples_split=5, n_estimators=100; accuracy: (test=0.715) f1: (test=0.510) precision: (test=0.587) recall: (test=0.451) total time=   0.2s
[CV 5/5; 118/128] START criterion=entropy, max_depth=50, min_samples_split=5, n_estimators=100
[CV 5/5; 118/128] END criterion=entropy, max_depth=50, min_samples_split=5, n_estimators=100; accuracy: (test=0.774) f1: (test=0.569) precision: (test=0.755) recall: (test=0.457) total time=   0.2s
[CV 1/5; 119/128] START criterion=entropy, max_depth=50, min_samples_split=5, n_estimators=200
[CV 1/5; 119/128] END criterion=entropy, max_depth=50, min_samples_split=5, n_estimators=200; accuracy: (test=0.755) f1: 

[CV 1/5; 124/128] END criterion=entropy, max_depth=50, min_samples_split=10, n_estimators=500; accuracy: (test=0.755) f1: (test=0.601) precision: (test=0.639) recall: (test=0.568) total time=   0.8s
[CV 2/5; 124/128] START criterion=entropy, max_depth=50, min_samples_split=10, n_estimators=500
[CV 2/5; 124/128] END criterion=entropy, max_depth=50, min_samples_split=10, n_estimators=500; accuracy: (test=0.755) f1: (test=0.479) precision: (test=0.800) recall: (test=0.341) total time=   0.8s
[CV 3/5; 124/128] START criterion=entropy, max_depth=50, min_samples_split=10, n_estimators=500
[CV 3/5; 124/128] END criterion=entropy, max_depth=50, min_samples_split=10, n_estimators=500; accuracy: (test=0.707) f1: (test=0.467) precision: (test=0.582) recall: (test=0.390) total time=   0.8s
[CV 4/5; 124/128] START criterion=entropy, max_depth=50, min_samples_split=10, n_estimators=500
[CV 4/5; 124/128] END criterion=entropy, max_depth=50, min_samples_split=10, n_estimators=500; accuracy: (test=0.70

In [24]:
# Build the model with best parameters and test
print(f"----- GRID SEARCH CV FOR RANDOM FOREST -----")
print(f"Best Parameters {rf_grid.best_params}")
print(f"F1 Score {rf_grid.f1}")
print(f"Precision Score {rf_grid.precision}")
print(f"Recall Score {rf_grid.recall}")
print(f"Accuracy Score {rf_grid.accuracy}")

----- GRID SEARCH CV FOR RANDOM FOREST -----
Best Parameters {'criterion': 'gini', 'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 50}
F1 Score 0.5536319652761479
Precision Score 0.6836250381776978
Recall Score 0.4782294489611562
Accuracy Score 0.747616271537764


### 3.5 Neural Networks

In [25]:
# search CV parameters for Random Forest 
nn_param_grid = {
    'hidden_layer_sizes': [(10, 10), (50, 50), (10, 50), (10, 10, 10), (50, 50, 50), (50, 10, 50), (10, 50, 10)],
    'activation': ['relu', 'logistic', 'tanh'],
    'solver': ['adam', 'sgd', 'lbfgs'],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': [0.01, 0.001],
    'max_iter': [50, 100, 500],
}


#### 3.5.1 Randomised Search CV  - Neural Network

In [26]:
# initialize the MLP neural network
nn_rand = RandomAndGridSearchCV(X_train=X_train_promapen,
                              y_train=y_train_promapen,
                              X_test=X_test_promapen,
                              y_test=y_test_promapen,
                              base_model_id=0,
                              classifier=MLPClassifier,
                              params=nn_param_grid)#### 3.2.1 Randomised Search CV

# Estimate best parameters for Neural Networks
nn_rand.estimate_best_params()

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5; 1/10] START activation=relu, hidden_layer_sizes=(10, 10, 10), learning_rate=invscaling, learning_rate_init=0.001, max_iter=500, solver=adam
[CV 1/5; 1/10] END activation=relu, hidden_layer_sizes=(10, 10, 10), learning_rate=invscaling, learning_rate_init=0.001, max_iter=500, solver=adam; accuracy: (test=0.731) f1: (test=0.562) precision: (test=0.597) recall: (test=0.531) total time=   0.4s
[CV 2/5; 1/10] START activation=relu, hidden_layer_sizes=(10, 10, 10), learning_rate=invscaling, learning_rate_init=0.001, max_iter=500, solver=adam
[CV 2/5; 1/10] END activation=relu, hidden_layer_sizes=(10, 10, 10), learning_rate=invscaling, learning_rate_init=0.001, max_iter=500, solver=adam; accuracy: (test=0.771) f1: (test=0.565) precision: (test=0.755) recall: (test=0.451) total time=   0.4s
[CV 3/5; 1/10] START activation=relu, hidden_layer_sizes=(10, 10, 10), learning_rate=invscaling, learning_rate_init=0.001, max_iter=500, 

[CV 2/5; 5/10] END activation=logistic, hidden_layer_sizes=(50, 50), learning_rate=adaptive, learning_rate_init=0.001, max_iter=100, solver=adam; accuracy: (test=0.747) f1: (test=0.471) precision: (test=0.757) recall: (test=0.341) total time=   0.3s
[CV 3/5; 5/10] START activation=logistic, hidden_layer_sizes=(50, 50), learning_rate=adaptive, learning_rate_init=0.001, max_iter=100, solver=adam
[CV 3/5; 5/10] END activation=logistic, hidden_layer_sizes=(50, 50), learning_rate=adaptive, learning_rate_init=0.001, max_iter=100, solver=adam; accuracy: (test=0.723) f1: (test=0.457) precision: (test=0.644) recall: (test=0.354) total time=   0.4s
[CV 4/5; 5/10] START activation=logistic, hidden_layer_sizes=(50, 50), learning_rate=adaptive, learning_rate_init=0.001, max_iter=100, solver=adam
[CV 4/5; 5/10] END activation=logistic, hidden_layer_sizes=(50, 50), learning_rate=adaptive, learning_rate_init=0.001, max_iter=100, solver=adam; accuracy: (test=0.727) f1: (test=0.558) precision: (test=0.5

[CV 3/5; 9/10] END activation=relu, hidden_layer_sizes=(50, 50), learning_rate=constant, learning_rate_init=0.01, max_iter=50, solver=adam; accuracy: (test=0.675) f1: (test=0.341) precision: (test=0.512) recall: (test=0.256) total time=   0.2s
[CV 4/5; 9/10] START activation=relu, hidden_layer_sizes=(50, 50), learning_rate=constant, learning_rate_init=0.01, max_iter=50, solver=adam
[CV 4/5; 9/10] END activation=relu, hidden_layer_sizes=(50, 50), learning_rate=constant, learning_rate_init=0.01, max_iter=50, solver=adam; accuracy: (test=0.723) f1: (test=0.549) precision: (test=0.592) recall: (test=0.512) total time=   0.2s
[CV 5/5; 9/10] START activation=relu, hidden_layer_sizes=(50, 50), learning_rate=constant, learning_rate_init=0.01, max_iter=50, solver=adam
[CV 5/5; 9/10] END activation=relu, hidden_layer_sizes=(50, 50), learning_rate=constant, learning_rate_init=0.01, max_iter=50, solver=adam; accuracy: (test=0.758) f1: (test=0.595) precision: (test=0.657) recall: (test=0.543) total

In [27]:
# Build the model with best parameters and test
print(f"----- RANDOMIZED SEARCH CV FOR NEURAL NETWORKS -----")
print(f"Best Parameters {nn_rand.best_params}")
print(f"F1 Score {nn_rand.f1}")
print(f"Precision Score {nn_rand.precision}")
print(f"Recall Score {nn_rand.recall}")
print(f"Accuracy Score {nn_rand.accuracy}")


----- RANDOMIZED SEARCH CV FOR NEURAL NETWORKS -----
Best Parameters {'solver': 'lbfgs', 'max_iter': 100, 'learning_rate_init': 0.001, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (10, 10), 'activation': 'relu'}
F1 Score 0.562389054113192
Precision Score 0.620634589924211
Recall Score 0.5198434206564289
Accuracy Score 0.7371550719005052


#### 3.5.2 Grid Search CV - Neural Network

In [28]:
nn_grid = RandomAndGridSearchCV(X_train=X_train_promapen,
                              y_train=y_train_promapen,
                              X_test=X_test_promapen,
                              y_test=y_test_promapen,
                              base_model_id=1,
                              classifier=MLPClassifier,
                              params=nn_param_grid)

# Estimate best parameters for Neural Networks
nn_grid.estimate_best_params()

Fitting 5 folds for each of 1134 candidates, totalling 5670 fits
[CV 1/5; 1/1134] START activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.01, max_iter=50, solver=adam
[CV 1/5; 1/1134] END activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.01, max_iter=50, solver=adam; accuracy: (test=0.751) f1: (test=0.575) precision: (test=0.646) recall: (test=0.519) total time=   0.1s
[CV 2/5; 1/1134] START activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.01, max_iter=50, solver=adam
[CV 2/5; 1/1134] END activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.01, max_iter=50, solver=adam; accuracy: (test=0.795) f1: (test=0.622) precision: (test=0.792) recall: (test=0.512) total time=   0.1s
[CV 3/5; 1/1134] START activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.01, max_iter=50, solver=adam
[CV 3/5; 1/113

[CV 3/5; 5/1134] END activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.01, max_iter=100, solver=sgd; accuracy: (test=0.699) f1: (test=0.453) precision: (test=0.564) recall: (test=0.378) total time=   0.1s
[CV 4/5; 5/1134] START activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.01, max_iter=100, solver=sgd
[CV 4/5; 5/1134] END activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.01, max_iter=100, solver=sgd; accuracy: (test=0.747) f1: (test=0.577) precision: (test=0.642) recall: (test=0.524) total time=   0.1s
[CV 5/5; 5/1134] START activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.01, max_iter=100, solver=sgd
[CV 5/5; 5/1134] END activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.01, max_iter=100, solver=sgd; accuracy: (test=0.786) f1: (test=0.595) precision: (test=0.780) recall: (test=0.

[CV 4/5; 9/1134] END activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.01, max_iter=500, solver=lbfgs; accuracy: (test=0.699) f1: (test=0.571) precision: (test=0.538) recall: (test=0.610) total time=   0.3s
[CV 5/5; 9/1134] START activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.01, max_iter=500, solver=lbfgs
[CV 5/5; 9/1134] END activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.01, max_iter=500, solver=lbfgs; accuracy: (test=0.742) f1: (test=0.590) precision: (test=0.613) recall: (test=0.568) total time=   0.3s
[CV 1/5; 10/1134] START activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.001, max_iter=50, solver=adam
[CV 1/5; 10/1134] END activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.001, max_iter=50, solver=adam; accuracy: (test=0.767) f1: (test=0.592) precision: (test=0.689) recall

[CV 2/5; 14/1134] END activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.001, max_iter=100, solver=sgd; accuracy: (test=0.703) f1: (test=0.178) precision: (test=1.000) recall: (test=0.098) total time=   0.1s
[CV 3/5; 14/1134] START activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.001, max_iter=100, solver=sgd
[CV 3/5; 14/1134] END activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.001, max_iter=100, solver=sgd; accuracy: (test=0.675) f1: (test=0.129) precision: (test=0.545) recall: (test=0.073) total time=   0.1s
[CV 4/5; 14/1134] START activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.001, max_iter=100, solver=sgd
[CV 4/5; 14/1134] END activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.001, max_iter=100, solver=sgd; accuracy: (test=0.691) f1: (test=0.206) precision: (test=0.667) recall

[CV 3/5; 18/1134] END activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.001, max_iter=500, solver=lbfgs; accuracy: (test=0.647) f1: (test=0.429) precision: (test=0.458) recall: (test=0.402) total time=   0.3s
[CV 4/5; 18/1134] START activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.001, max_iter=500, solver=lbfgs
[CV 4/5; 18/1134] END activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.001, max_iter=500, solver=lbfgs; accuracy: (test=0.699) f1: (test=0.571) precision: (test=0.538) recall: (test=0.610) total time=   0.3s
[CV 5/5; 18/1134] START activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.001, max_iter=500, solver=lbfgs
[CV 5/5; 18/1134] END activation=relu, hidden_layer_sizes=(10, 10), learning_rate=constant, learning_rate_init=0.001, max_iter=500, solver=lbfgs; accuracy: (test=0.742) f1: (test=0.590) precision: (test=0.6

...

[CV 2/5; 1117/1134] END activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.01, max_iter=50, solver=adam; accuracy: (test=0.755) f1: (test=0.520) precision: (test=0.733) recall: (test=0.402) total time=   0.1s
[CV 3/5; 1117/1134] START activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.01, max_iter=50, solver=adam
[CV 3/5; 1117/1134] END activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.01, max_iter=50, solver=adam; accuracy: (test=0.679) f1: (test=0.494) precision: (test=0.513) recall: (test=0.476) total time=   0.1s
[CV 4/5; 1117/1134] START activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.01, max_iter=50, solver=adam
[CV 4/5; 1117/1134] END activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.01, max_iter=50, solver=adam; accuracy: (test=0.671) f1: (test=0.523) preci

[CV 4/5; 1121/1134] END activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.01, max_iter=100, solver=sgd; accuracy: (test=0.747) f1: (test=0.588) precision: (test=0.634) recall: (test=0.549) total time=   0.1s
[CV 5/5; 1121/1134] START activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.01, max_iter=100, solver=sgd
[CV 5/5; 1121/1134] END activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.01, max_iter=100, solver=sgd; accuracy: (test=0.778) f1: (test=0.580) precision: (test=0.760) recall: (test=0.469) total time=   0.1s
[CV 1/5; 1122/1134] START activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.01, max_iter=100, solver=lbfgs
[CV 1/5; 1122/1134] END activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.01, max_iter=100, solver=lbfgs; accuracy: (test=0.707) f1: (test=0.523) p

[CV 5/5; 1125/1134] END activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.01, max_iter=500, solver=lbfgs; accuracy: (test=0.690) f1: (test=0.503) precision: (test=0.527) recall: (test=0.481) total time=   2.1s
[CV 1/5; 1126/1134] START activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.001, max_iter=50, solver=adam
[CV 1/5; 1126/1134] END activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.001, max_iter=50, solver=adam; accuracy: (test=0.759) f1: (test=0.589) precision: (test=0.662) recall: (test=0.531) total time=   0.1s
[CV 2/5; 1126/1134] START activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.001, max_iter=50, solver=adam
[CV 2/5; 1126/1134] END activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.001, max_iter=50, solver=adam; accuracy: (test=0.763) f1: (test=0.512)

[CV 2/5; 1130/1134] END activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.001, max_iter=100, solver=sgd; accuracy: (test=0.703) f1: (test=0.315) precision: (test=0.654) recall: (test=0.207) total time=   0.2s
[CV 3/5; 1130/1134] START activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.001, max_iter=100, solver=sgd
[CV 3/5; 1130/1134] END activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.001, max_iter=100, solver=sgd; accuracy: (test=0.703) f1: (test=0.413) precision: (test=0.591) recall: (test=0.317) total time=   0.2s
[CV 4/5; 1130/1134] START activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.001, max_iter=100, solver=sgd
[CV 4/5; 1130/1134] END activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.001, max_iter=100, solver=sgd; accuracy: (test=0.711) f1: (test=0.556) 

[CV 3/5; 1134/1134] END activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.001, max_iter=500, solver=lbfgs; accuracy: (test=0.655) f1: (test=0.533) precision: (test=0.480) recall: (test=0.598) total time=   1.6s
[CV 4/5; 1134/1134] START activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.001, max_iter=500, solver=lbfgs
[CV 4/5; 1134/1134] END activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.001, max_iter=500, solver=lbfgs; accuracy: (test=0.651) f1: (test=0.466) precision: (test=0.469) recall: (test=0.463) total time=   1.7s
[CV 5/5; 1134/1134] START activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.001, max_iter=500, solver=lbfgs
[CV 5/5; 1134/1134] END activation=tanh, hidden_layer_sizes=(10, 50, 10), learning_rate=adaptive, learning_rate_init=0.001, max_iter=500, solver=lbfgs; accuracy: (test=0.690) f1: (te

In [29]:
# Build the model with best parameters and test
print(f"----- GRID SEARCH CV FOR NEURAL NETWORKS -----")
print(f"Best Parameters {nn_grid.best_params}")
print(f"F1 Score {nn_grid.f1}")
print(f"Precision Score {nn_grid.precision}")
print(f"Recall Score {nn_grid.recall}")
print(f"Accuracy Score {nn_grid.accuracy}")


----- GRID SEARCH CV FOR NEURAL NETWORKS -----
Best Parameters {'activation': 'tanh', 'hidden_layer_sizes': (10, 50), 'learning_rate': 'constant', 'learning_rate_init': 0.01, 'max_iter': 100, 'solver': 'adam'}
F1 Score 0.5921299662209685
Precision Score 0.6182316372932816
Recall Score 0.57353206865402
Accuracy Score 0.740374400829123


## 4. Results

In [30]:
models = {"LG-RAND": lg_rand, "LG-GRID": lg_grid, 
          "SVM-RAND": svm_rand, "SVM-GRID": svm_grid, 
          "DT-RAND": dt_rand, "DT-GRID": dt_grid, 
          "RF-RAND": rf_rand, "RF-GRID": rf_grid, 
          "NN-RAND": nn_rand, "NN-GRID": nn_grid}

result = []
for name, model in models.items():
    result.append([name, model.accuracy, model.f1, model.recall, model.precision, model.best_params])

result_df = pd.DataFrame(result, columns=["Name", "Accuracy", "F1-Score", "Recall", "Precision", "Best Params"])
result_df = result_df.round({"Accuracy": 4, "F1-Score": 4, "Recall":4, "Precision": 4})
result_df[["Name", "F1-Score", "Recall", "Precision", "Accuracy"]]

,Name,F1-Score,Recall,Precision,Accuracy
0,LG-RAND,0.5576,0.4878,0.6695,0.7452
1,LG-GRID,0.5624,0.4854,0.6805,0.7524
2,SVM-RAND,0.5436,0.5246,0.5787,0.7162
3,SVM-GRID,0.5436,0.5246,0.5787,0.7162
4,DT-RAND,0.5044,0.4366,0.6067,0.7203
5,DT-GRID,0.5076,0.5247,0.4955,0.6672
6,RF-RAND,0.5446,0.4709,0.6667,0.7444
7,RF-GRID,0.5536,0.4782,0.6836,0.7476
8,NN-RAND,0.5624,0.5198,0.6206,0.7372
9,NN-GRID,0.5921,0.5735,0.6182,0.7404


In [31]:
# Save the results for plotting graph and persisting results
result_df.to_csv("results/ProMapEn/result-10-models.csv", header=True, index=False)